In [118]:
a = [0,0,0,0,0]

In [119]:
n = 7
a = [ 0 for i in range(n)]
a

[0, 0, 0, 0, 0, 0, 0]

In [122]:
n=7
a = [ [0 for i in range(n)]] # 2차원 배열로 보면 원소 개수가 1개이다.
a

[[0, 0, 0, 0, 0, 0, 0]]

In [123]:
n = 7
a = [ [0 for i in range(n)]] # 2차원 배열로 보면 원소 개수가 1개이다.
a
len(a) # 이건 값이 1

1

In [124]:
n = 7
a = [ [0 for i in range(n)]] # 2차원 배열로 보면 원소 개수가 1개이다.
a

len(a[0])

7

In [10]:
n=5
a = [ [0 for i in range(n)] for j in range(n)]
a
#len(a[0])

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [127]:
# 더 쉬운 방법
n=5
a = [ [0]*n for j in range(n)]
a

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [131]:
a = [0]* n
print(a)
b = [[0]*n]
print(b)

[0, 0, 0, 0, 0]
[[0, 0, 0, 0, 0]]


In [133]:
#알고리즘적인 문제
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까

for i in range(n):
    col += 1
    a[row][col] = count
    count += 1
a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [140]:
# &&&&&&& 실험용 &&&&&&&&&&&
#알고리즘적인 문제
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = 0 # 행 ( **** 왜 근데 원래 식에서는 -1로 주고 밑에도 col의 위치가 다르지?****)
count = 1 #1부터 증가하니까   ( col에 0을 주고 아래에서도 col을 먼저 더해주지 않으면 list의 range를 벗어나는 값이 나옴)

for i in range(n):
    a[row][col] = count
    col += 1
    count += 1

a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [15]:
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수

for i in range(num):
    col += 1
    a[row][col] = count
    count += 1
    
num -= 1

for i in range(num):
    row += 1
    a[row][col] = count
    count += 1

a

[[1, 2, 3, 4, 5],
 [0, 0, 0, 0, 6],
 [0, 0, 0, 0, 7],
 [0, 0, 0, 0, 8],
 [0, 0, 0, 0, 9]]

In [145]:
# &&&&&&& 실험용 &&&&&&&&&&&
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수

for i in range(num):
    
    a[row][col] = count  #( 여기서 col =-1 로 시작하고 위치를 아래에 두니 count=1 의 값이 count=6의 값으로 덮어씌워짐.)
    col += 1
    count += 1
    
num -= 1

for i in range(num):
    
    a[row][col] = count
    row += 1
    count += 1

a
#print(a[0][-1])

[[2, 3, 4, 5, 6],
 [0, 0, 0, 0, 7],
 [0, 0, 0, 0, 8],
 [0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0]]

In [28]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()

n=11
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수


while True:
    for i in range(num):
        col += 1
        a[row][col] = count
        count += 1

    num -= 1 #회전수 감소할 때 소환
    if num == 0:
        break
        
    for i in range(num):
        row += 1 # 행건드릴때
        a[row][col] = count
        count += 1

    for i in range(num):
        col -= 1 #열 건드릴때
        a[row][col] = count
        count += 1

    num -= 1
    if num == 0:
        break
        
    for i in range(num):
        row -= 1
        a[row][col] = count
        count += 1

display(a)

   1   2   3   4   5   6   7   8   9  10  11
  40  41  42  43  44  45  46  47  48  49  12
  39  72  73  74  75  76  77  78  79  50  13
  38  71  96  97  98  99 100 101  80  51  14
  37  70  95 112 113 114 115 102  81  52  15
  36  69  94 111 120 121 116 103  82  53  16
  35  68  93 110 119 118 117 104  83  54  17
  34  67  92 109 108 107 106 105  84  55  18
  33  66  91  90  89  88  87  86  85  56  19
  32  65  64  63  62  61  60  59  58  57  20
  31  30  29  28  27  26  25  24  23  22  21


In [146]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
# 코드가 부호 제외하고는 같아서 반전코드 사용해서 코드 단순화
n=5
a = [ [0]*n for j in range(n)]
row = 0 # 열
col = -1 # 행
count = 1 #1부터 증가하니까
num = n # 회전수 담당 변수
sign = 1

while True:
    for i in range(num):
        col += sign #sign 반전코드 만드는것.
        a[row][col] = count
        count += 1

    num -= 1
    if num == 0:
        break
        
    for i in range(num):
        row += sign # 행건드릴때
        a[row][col] = count
        count += 1
    sign = - sign #sign 부호반전이 while 루프 돌면서 발생

display(a)

   1   2   3   4   5
  16  17  18  19   6
  15  24  25  20   7
  14  23  22  21   8
  13  12  11  10   9


In [39]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n):
            a[i][j] = count
            count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   2   3   4   5
   6   7   8   9  10
  11  12  13  14  15
  16  17  18  19  20
  21  22  23  24  25


In [41]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n-i):
            a[i+j][i] = count
            count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   0   0   0   0
   2   6   0   0   0
   3   7  10   0   0
   4   8  11  13   0
   5   9  12  14  15


In [45]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n-i):
            a[i][n-i-1-j] = count  
            count += 1
    

n=5
a = init(n)
fill(a,n)
display(a)

   5   4   3   2   1
   9   8   7   6   0
  12  11  10   0   0
  14  13   0   0   0
  15   0   0   0   0


In [230]:
# &&&&&&&&&&&&실험용 &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n-i):
            a[i][n-i-j-1] = count 
            count += 1
    

n=5
a = init(n)
fill(a,n)
display(a)

   5   4   3   2   1
   9   8   7   6   0
  12  11  10   0   0
  14  13   0   0   0
  15   0   0   0   0


In [54]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j) == k:
                    a[i][j] = count
                    count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   2   4   7  11
   3   5   8  12  16
   6   9  13  17  20
  10  14  18  21  23
  15  19  22  24  25


In [56]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j) == k:
                    a[j][i] = count
                    count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   3   6  10  15
   2   5   9  14  19
   4   8  13  18  22
   7  12  17  21  24
  11  16  20  23  25


In [67]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    inc = 0
    flag = 1
    for i in range(n):
        for j in range(n-inc*2):
            a[inc+j][i] = count
            count += 1
        if n//2 == i:
            flag = -flag
        inc += flag
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   0   0   0  13
   2   6   0  10  14
   3   7   9  11  15
   4   8   0  12  16
   5   0   0   0  17


## 클래스 만들기 

In [69]:
class FourCal:
    def add(self, a, b):
        result = a + b
        return result

a = FourCal()
print(a.add(3,4))


7


In [8]:
class FourCal:
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result

a = FourCal()
a.setdata(3,4)
print(a.add())

# self.first = 3, self.second = 4 가 전달되어 setdata메소드 수행
# self는 전달된 객체 a이므로 a.first = 3, a.second =4 로 해석됨
# a.first = 3 문장이 수행되면 a객체에 객체변수 first가 생성되고 값 4가 저장됨
#마찬가지로 a.second = 4

#클래스로 만든 객체의 객체변수는 다른 객체의 객체변수에 상관없이 독립적인 값 유지


# FourCal.setdata(a, 3, 4) 클래스이름.메서드 형태로 호출하는 것도 가능하다
# 하지만 이 경우 객체 a를 매개변수 self에 꼭 전달해줘야 한다.
# but 객체.메서드 형태로 호출할 때는 self 반드시 생략해서 호출

7


###  생성자
객체에 초깃값을 설정해야 할 필요가 있을 때는 생성자를 구현하는 것이 안전한 방법이다.
* 생성자(constructor)란 객체가 생성될 때 자동으로 호출되는 메서드를 의미한다. 
파이썬 메서드 이름으로 \_\_init\_\_ 를 사용하면 이 메서드는 생성자가 된다

* 생성자를 사용하면 객체가 생성될 때 객체의 속성을 생성 및 초기화 할 수 있다

In [10]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    # __init__ 메서드는 setdata메서드와 이름만 다르고 모든게 동일하다
    # 단 메서드 이름을 __init__으로 했기 때문에 생성자로 인식되어
    # 객체가 생성되는 시점에 자동으로 호출되는 차이가 있다.
    # a = FourCal()을 생성자가 있는 상태에서 호출하면
    # 오류가 난다. 생성자의 매개변수 first와 second에 해당하는 값이 전달되지
    # 않았기 때문. a = FourCal(3,2)라고 해야 오류 나지 않는다
    # 이렇게 하면 __init__메서드의 매개변수에는
    # self -> 생성되는 객체, first ->  3, second->2 값이 대입된다
    
    def setdata(self, first, second): #메서드 (클래스 안에 구현된 함수)
        self.first = first #메서드의 첫번째 매개변수 self에는 setdata메서드를
                            #호출한 객체 a가 자동으로 전달된다
        self.second = second
        # 파이썬 메서드의 첫번째 매개변수 이름은 관례적으로 self를 사용
        # 객체를 호출할 때 호출한 객체 자신이 전달되기 때문에 self를 사용한것
    
    def add(self):
        result = self.first + self.second
        return result

a = FourCal(3,4)
print(a.add())

FourCal.__init__()
7


###  클래스의 상속
상속(Inheritance)이란 물려받다라는 뜻으로 어떤 클래스를 만들 때 다른 클래스의 기능을 물려받을 수 있게 만드는 것이다

* 클래스를 상속하기 위해서는   
*class 클래스 이름 (상속할 클래스 이름)**

* 상속받은 클래스는 상속한 클래스의 모든 기능을 사용할 수 있어야 한다
* 상속은 기존 클래스를 변경하지 않고 기능을 추가하거나 기존 기능을 변경하려고 할 때 사용한다

In [11]:
class MoreFourCal(FourCal): # FourCal을 상속한다
    def pow(self):
        result = self.first ** self.second
        return result

a = MoreFourCal(3,4)
a.pow()

FourCal.__init__()


81

In [2]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result
    
    def div(self):
        print("FourCal.div()")
        result = self.first/self.second
        return result

a = FourCal(3,4)
print(a.div())

FourCal.__init__()
FourCal.div()
0.75


In [3]:
class FourCal:
    def __init__(self, first, second):#생성자
        print("FourCal.__init__()")
        self.first = first
        self.second = second
    
    def setdata(self, first, second):
        self.first = first
        self.second = second
    
    def add(self):
        result = self.first + self.second
        return result
    
    def div(self):
        print("FourCal.div()")
        result = self.first/self.second
        return result

a = FourCal(3,0)
print(a.div())

FourCal.__init__()
FourCal.div()


ZeroDivisionError: division by zero

In [12]:
class MoreFourCal(FourCal):
    def pow(self):
        result = self.first ** self.second
        return result
    
    def div(self): #오버라이딩. 부모랑 똑같은 이름의 함수 호출하면 
                    #자식의 함수가 호출됨
        print("MoreFourCal.div()")
        if(self.second == 0):  # 나누는 값이 0인 경우 숫자 을 돌려주도록 수정
            print('분모는 0으로 지정하면 안되요')
            return
        
        result = self.first/self.second
        return result
# 이렇게 부모클래스에 있는 메서드를 동일한 이름으로 다시 만드는 것을
# 메서드 오버라이딩이라고한다. 
a = MoreFourCal(3,0)
a.div()

FourCal.__init__()
MoreFourCal.div()
분모는 0으로 지정하면 안되요


In [4]:
def foo():
    print('foo()')
    
def foo(a):
    print('foo(a)')
    
foo()
foo(3) #이건 오버로딩인데 파이썬에서는 지원하지 않는다. 전통적인 객체지향 프로그램은 오버로딩 지원함.
#호출안하면 에러 안난다. 
#위에거 호출하면 에러난다. 
#foo 가 변수같은 존재
#함수도 파이썬에서는 객체라 최신의 함수만 남아 작동하게 된다.

TypeError: foo() missing 1 required positional argument: 'a'

### 클래스 변수 
* 객체변수는 다른 객체들에 영향받지 않고 독립적으로 그 값을 유지한다
* 클래스변수의 경우 클래스 안에 함수를 선언하는 것과 마찬가지로 클래스안에 
변수를 선언하여 생성한다

In [25]:
class Family: 
    lastname = '김'#지역변수와 멤버변수 사이에 있는 희안한 존재, 
                    #클래스안에만 있다. 클래스 변수. 
                    #멤버변수가 아니다. 파이썬은 클래스도 객체다.
    def foo():
        name = '홍길동' #local 변수
        
print(Family.lastname)

김


In [26]:
a = Family()
print(a.lastname)

김


In [27]:
b = Family()
print(b.lastname)

김


In [28]:
Family.lastname = '박' # 클래스 전체에 소속된 변수이기 때문에 
                       #수정하면 통으로 수정됨

In [29]:
print(a.lastname)

박


In [30]:
print(b.lastname)  
# 클래스 변수는 클래스로 만든 모든 객체에 공유된다는 특징

박


In [114]:
id(Family.lastname)

2378593582336

In [115]:
id(a.lastname)

2378593582336

In [117]:
id(b.lastname)  # 모두 같은 메모리를 가리키고 있다.

2378593582336

### 모듈
모듈이란 함수나 변수 또는 클래스를 모아 놓은 파일이다. 모듈은 다른 파이썬 프로그램에서 불러와 사용할 수 있게끔 만든 파이썬 파일이라고도 할 수 있다.


In [156]:
import mod1
print(mod1.add(3,2))
print(mod1.sub(3,3))

mod1: add()
5
mod1: sub()
0


In [158]:
from mod1 import add  # 이렇게 파일에서 함수를 임포트 한다면 모듈네임을 타입하지 않아도 된다.
print(add(3,1))

mod1: add()
4


In [162]:
from mod1 import add,sub 
print(add(3,1))
print(sub(3,1))

mod1: add()
4
mod1: sub()
2


In [160]:
from mod1 import * # 이렇게 별표 치면 모든 함수 다 임포트
print(add(3,1))
print(sub(3,1))

mod1: add()
4
mod1: sub()
2


In [161]:
print(__name__)  #자기사진의 이름은 항상 main으로 나온다.

__main__


In [163]:
from mod1 import *
add(1,2)

mod1: add()


3

In [164]:
from mod1 import *
print(mod1.__name__)

mod1


### 예외처리 
파이썬은 오류가 발생하면 프로그램을 중단하고 오류 메시지를 보여준다

In [166]:
open("나없는파일1", "r") # 없는 파일을 열려고 시도하면 FileNotFoundError 발생

FileNotFoundError: [Errno 2] No such file or directory: '나없는파일1'

In [168]:
try : #정상적으로 시도되는 expression
    4 / 0 # 0으로 나누려니까 ZeroDivisionError 발생
except ZeroDivisionError as e:
    print(e)

division by zero


In [31]:
f = open('foof.txt', 'w')

try : 
    4 / 0
finally: #예외발생하더라도 무조건 실행해야 하는것은 finally블록에 넣으면 된다.
    f.close()
    
    print(e)

NameError: name 'e' is not defined

In [34]:
a = [1,2,3]
a[4]
# a[4]는 a 리스트에서 얻을 수 없는 값이다. 따라서 IndexError 발생

IndexError: list index out of range

### 오류 예외 처리 기법
try, except문 : try블록 수행 중 오류가 발생하면  except블록이 수행된다. 하지만 try블록에서 오류가 발생하지 않는다면 except블록은 수행되지 않는다

``` 
try: 
       ...
except [발생오류[as 오류 메시지 변수]]:
        ...
        
```


### 여러개의 오류 처리하기 

In [175]:
try :
    4/0
    a = [1, 2]
    a[3] = 4
    
    
except (ZeroDivisionError, IndexError) as e:
    print(e)


division by zero


In [176]:
try :
    
    a = [1, 2]
    a[3] = 4
    4/0
    
except (ZeroDivisionError, IndexError) as e:
    print(e)


list assignment index out of range


### 오류 회피하기 

In [181]:
try :
    f = open("나없는파일3", "r")
except FileNotFoundError:
    pass  #아무것도 안나타나게 함

### 오류 일부러 발생시키기 

In [207]:
class Bird:
    def fly(self):
        raise NotImplementedError
        
b = Bird()
b.fly()

NotImplementedError: 

In [232]:
class Bird:
    def fly(self):
        raise NotImplementedError
        

class Eagle(Bird):
    #def fly(self):
        pass
    
e = Eagle()
e.fly()

NotImplementedError: 

In [191]:
class MyError(Exception):
    pass

def say_nick(nick):
    if(nick == '멍충이'):
        raise MyError()
    print(nick)
    
say_nick("호랭이")

호랭이


In [194]:
try:
    say_nick("멍충이")

except MyError:
    print('허용되지 않은 별명입니다.')

허용되지 않은 별명입니다.


In [202]:
class MyError(Exception):
    def __str__(self):
        return "허용되지 않는 별명입니다."

def say_nick(nick):
    if(nick == '멍충이'):
        raise MyError()
    print(nick)
    
try:
    say_nick("멍충이")

except MyError as e:
    print(e)

허용되지 않는 별명입니다.


### 내장 함수 

In [210]:
for value in ['body', 'foo', 'bar']:
    print(value)

body
foo
bar


In [213]:
for i, value in enumerate (['body', 'foo', 'bar']):
    print(i, value)

0 body
1 foo
2 bar


In [214]:
a = ['body', 'fpp', 'bar']
b = [100, 200 , 300]

for i in range(3):
    print(a[i], b[i])

body 100
fpp 200
bar 300


In [216]:
a = ['body', 'fpp', 'bar']
b = [100, 200 , 300, 400]

for i in range(4):
    print(a[i], b[i])

body 100
fpp 200
bar 300


IndexError: list index out of range

In [221]:
a = ['body', 'fpp', 'bar']
b = [100, 200 , 300, 400]

for temp in zip(a, b):
    print(temp)

('body', 100)
('fpp', 200)
('bar', 300)


In [226]:
list ( zip([1,2,3], [4,5,6]))

[(1, 4), (2, 5), (3, 6)]

In [223]:
list ( zip([1,2,3], [4,5,6], [7,8,9]))

[(1, 4, 7), (2, 5, 8), (3, 6, 9)]

In [224]:
list ( zip("abc", "def"))

[('a', 'd'), ('b', 'e'), ('c', 'f')]

In [225]:
list ( zip("abc", [1,2,3,]))

[('a', 1), ('b', 2), ('c', 3)]

In [233]:
f = open("nofile", 'r')

FileNotFoundError: [Errno 2] No such file or directory: 'nofile'

## 5장 연습문제 

### 연습문제 1

In [234]:
class Calculator:
    def __init__(self):
        self.value = 0
        
    def add(self,val):
        self.value += val

In [235]:
class UpgradeCalculator(Calculator):
    def minus(self,val):
        self.value -= val
        


In [236]:
cal = UpgradeCalculator()
cal.add(10)
cal.minus(7)

print(cal.value)

3


### 연습문제 2 

In [247]:
class Calculator:
    def __init__(self):
        self.value = 0
        
    def add(self,val):
        self.value += val
        
class MaxLimitCalculator(Calculator):
    def add(self,val):
        self.value += val     
        if(cal.value > 100):
            cal.value = 100
    
    


cal = MaxLimitCalculator()
cal.add(50)
cal.add(60)

print(cal.value)
    
    

100


In [248]:
all([1,2,abs(-3)-3])

False

In [249]:
chr(ord('a')) == 'a'

True

In [264]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for i in range(n):
        for j in range(n):
            a[n-i-5][n-3-j] = count
            count += 1
            if ((n-i-5) == -1):
                (n-i-5) == 4
    
n=5
a = init(n)
fill(a,n)
display(a)

   3   2   1   5   4
  23  22  21  25  24
  18  17  16  20  19
  13  12  11  15  14
   8   7   6  10   9


In [54]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        

def init(n):
    a = [ [0]*n for j in range(n)]
    return a

def fill(a, n):
    count = 1
    for k in range(2*n-1):
        for i in range(n):
            for j in range(n):
                if (i+j) == k:
                    a[i][j] = count
                    count += 1
    
    
n=5
a = init(n)
fill(a,n)
display(a)

   1   2   4   7  11
   3   5   8  12  16
   6   9  13  17  20
  10  14  18  21  23
  15  19  22  24  25


In [268]:
def display(a):
    for i in range(n):
        for j in range(n):
            print("%4d" % a[i][j], end='')
        print()


# 코드가 부호 제외하고는 같아서 반전코드 사용해서 코드 단순화
n = 5
a = [[0] * n for j in range(n)]
row = 0  # 열
col = -1  # 행
count = 1  # 1부터 증가하니까
num = n  # 회전수 담당 변수


while True:
    for i in range(num):
        col += 1
        a[row][col] = count
        count += 1

    num -= 1
    if num == 0:
        break

    for i in range(num):
        row += 1  # 행건드릴때
        a[row][col] = count
        count += 1


display(a)

IndexError: list assignment index out of range

## 홀수마방진 
### 1. 1은 첫번째 행의 가운데에 채운다
### 2. 다음 숫자는 왼쪽 열의 윗쪽 행에 채운다
### 3. 행을 넘어가면 마지막 행에 채운다
### 4. 열을 넘어가면 마지막 열에 채운다
### 5. 채울 공간에 이미 숫자가 채워져 있다면 현재 위치의 아랫쪽 행에 채운다(현재 위치를 기억하고 있어야 함)
### # 검산도 해야한다 (가로/ 세로 / 대각선 합 확인)


#### 1) 전체 회전수가 중요하다
#### 2) 1의 시작점을 찍고 시작해야 한다
* x,y는 좌표계, [x][y]는 행렬이다
* 하지만 컴퓨터는 왼쪽 상단을 0,0으로 갖으며 이는 제 4 사분면을 사용한다는 의미이다. 
* 그렇기 때문에 우리는 row와 col을 사용하자

In [1]:
def  display(a, n):
    for i in range(n):
        for j in range(n):
            print("%4d"%a[i][j], end='')
        print()
        
def init(n):
    a = [ [0]*n for j in  range(n) ] 
    return a

def fill(a, n):
    row=0
    count=1
    col=n//2 #모든 수의 가운데는 n//2이다. 몫이 2이기 때문이다.
    r=0 # 여기서 r과 c는 임시변수다. 
    c=0
    a[row][col] = count # 결국 이것을 for문 밖에 둔 이유는 
                 #0,2에서 1이 시작해야 하기 때문임을 못 박는 것이다.

    for i in range(n*n-1): # 회전수-1이유 > 위에서 이미 1은 [0][2]에 고정
        count += 1 # for문에서 회전 시작 전에 count 에 1 더하고 시작
        r = row-1
        c = col-1
        if( r < 0 ):
            r = n-1 # 마지막행으로 보낸다는 뜻

        if( c < 0 ):
            c = n-1 # 마지막열로 보낸다는 뜻

        if( a[r][c] > 0 ): # 이미 값이 배열되어 있는 경우
            r = row+1
            c = col

        a[r][c] = count
        row = r #회전 마치고 난 값을 현재 값으로 재설정해줌
        col = c

def check(a, n): #가로, 세로 , 대각선의 합으로 검사해주는 함수
    for i in range(n):
        for j in range(n):
            a[i][n] += a[i][j] #n을 넣어주면 해당 열에 합으로 채워짐
            a[n][i] += a[j][i] #n을 넣어주면 해당 행에 합으로 채워짐
            if( i == j ): # 대각선 합의 경우( 단 오른쪽 아래의 값 하나 구한것)
                a[n][n] += a[i][j]
    
n=5
a = init(n+1) # 검사한 합이 와야 함으로 추가 행렬을 만들어줌
fill(a, n) # 채워넣는 값은 5X5이므로 n 그대로
check(a, n) #검사하는 부분은 5X5 부분임으로 n 그대로
display(a,n+1) # 검산까지 표시해야 함으로 6X6

  15   8   1  24  17  65
  16  14   7   5  23  65
  22  20  13   6   4  65
   3  21  19  12  10  65
   9   2  25  18  11  65
  65  65  65  65  65  65
